<a href="https://colab.research.google.com/github/lgiesen/forest_height/blob/main/notebooks/PrepareDataForCnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [101]:
import numpy as np
import pandas as pd
import datetime, os, cv2
from matplotlib import pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae, mean_absolute_percentage_error as mape
from keras.models import Sequential, load_model
from keras.layers import Dense, BatchNormalization, Dropout, InputLayer, Flatten, Conv2D, MaxPool2D
from keras.callbacks import TensorBoard, ModelCheckpoint

In [ ]:
!unzip gdrive/My\ Drive/DataDa2/images_train.zip #first part of the data

In [ ]:
!unzip gdrive/My\ Drive/DataDa2/masks_train.zip

## Load all the data and extract all features and labes

In [5]:
size = 5 #define window size should be odd so that the label is in the middle
shape = (10, size, size) #define shape of features
labels1 = np.ones(1) #array for labels
data1 = np.ones(shape) #array for features
data1 = np.expand_dims(data1, axis=0) #expand dimension to concatenate
for j in range(20): #iterate over images in directory
  if j < 10:
    X = np.load('/content/images/image_00'+ str(j) + '.npy')
    y = np.load('/content/masks/mask_00'+ str(j) + '.npy')
    indices = np.argwhere(y > 0) #select all values with label
    indices_2d = indices[:, 1:] #extract indices
    ind_y = np.ones(2).reshape(-1,2) #array to collect indices
    for i in indices_2d: #iterate over indices
      if shape == X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1].shape: #select only features with the same shape because of labels at the image border
        temp = X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1] #save them temporary
        temp2 = np.expand_dims(temp, axis=0) #expand dimension to concatenate
        data1 = np.concatenate((data1, temp2), axis=0) #concatenation

        ind_y = np.concatenate((ind_y, i.reshape(-1,2)), axis=0) #concatenation of index so that they have the same order and length as the features

    ind_y = ind_y[1:] #remove first dummy values
    indices_1 = ind_y[:, 0].astype(int)
    indices_2 = ind_y[:, 1].astype(int)
    data_y = y[0, indices_1, indices_2] #extract labels
    labels1 = np.concatenate((labels1, data_y), axis = 0) #concatenate labels

  if j >= 10:
    X = np.load('/content/images/image_0'+ str(j) + '.npy')
    y = np.load('/content/masks/mask_0'+ str(j) + '.npy')
    indices = np.argwhere(y > 0)
    indices_2d = indices[:, 1:]
    ind_y = np.ones(2).reshape(-1,2)
    for i in indices_2d:
      if shape == X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1].shape:
        temp = X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1]
        temp2 = np.expand_dims(temp, axis=0)
        data1 = np.concatenate((data1, temp2), axis=0)

        ind_y = np.concatenate((ind_y, i.reshape(-1,2)), axis=0)

    ind_y = ind_y[1:]
    indices_1 = ind_y[:, 0].astype(int)
    indices_2 = ind_y[:, 1].astype(int)
    data_y = y[0, indices_1, indices_2]
    labels1 = np.concatenate((labels1, data_y), axis = 0)

data1 = data1[1:] #remove first dummy values
labels1 = labels1[1:] #remove first dummy values

In [ ]:
!unzip gdrive/My\ Drive/DataDa22/images_02.zip #second part of the data

In [ ]:
!unzip gdrive/My\ Drive/DataDa22/masks_02.zip

In [9]:
size = 5
shape = (10, size, size)
labels2 = np.ones(1)
data2 = np.ones(shape)
data2 = np.expand_dims(data2, axis=0)
for j in range(20):
  if j < 10:
    X = np.load('/content/images/image_00'+ str(j) + '.npy')
    y = np.load('/content/masks/mask_00'+ str(j) + '.npy')
    indices = np.argwhere(y > 0)
    indices_2d = indices[:, 1:]
    ind_y = np.ones(2).reshape(-1,2)
    for i in indices_2d:
      if shape == X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1].shape:
        temp = X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1]
        temp2 = np.expand_dims(temp, axis=0)
        data2 = np.concatenate((data2, temp2), axis=0)

        ind_y = np.concatenate((ind_y, i.reshape(-1,2)), axis=0)

    ind_y = ind_y[1:]
    indices_1 = ind_y[:, 0].astype(int)
    indices_2 = ind_y[:, 1].astype(int)
    data_y = y[0, indices_1, indices_2]
    labels2 = np.concatenate((labels2, data_y), axis = 0)

  if j >= 10:
    X = np.load('/content/images/image_0'+ str(j) + '.npy')
    y = np.load('/content/masks/mask_0'+ str(j) + '.npy')
    indices = np.argwhere(y > 0)
    indices_2d = indices[:, 1:]
    ind_y = np.ones(2).reshape(-1,2)
    for i in indices_2d:
      if shape == X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1].shape:
        temp = X[:, i[0] - (size//2):i[0] + (size//2) + 1, i[1] - (size//2):i[1] + (size//2) + 1]
        temp2 = np.expand_dims(temp, axis=0)
        data2 = np.concatenate((data2, temp2), axis=0)

        ind_y = np.concatenate((ind_y, i.reshape(-1,2)), axis=0)

    ind_y = ind_y[1:]
    indices_1 = ind_y[:, 0].astype(int)
    indices_2 = ind_y[:, 1].astype(int)
    data_y = y[0, indices_1, indices_2]
    labels2 = np.concatenate((labels2, data_y), axis = 0)

data2 = data2[1:]
labels2 = labels2[1:]

In [100]:
features = np.concatenate((data1, data2), axis = 0) #concatenate both parts
labels = np.concatenate((labels1, labels2), axis = 0)

In [13]:
print(features.shape)
print(labels.shape)

(38531, 10, 5, 5)
(38531,)


## Scale data

In [102]:
ceiling = 2000
features2 = features
features2[features2 > ceiling] = ceiling
features2 = features2/ceiling
labels2 = labels/ceiling

In [147]:
ceiling = 2000
features3 = features
features3[features3 > ceiling] = ceiling
features3 = (features3 - np.mean(features3)) / np.std(features3)** (1/2) #normalize features
labels3 = (labels - np.mean(labels)) / np.std(labels)** (1/2)

## Create equally distributed training *data*

In [153]:
sample_size = 800

num = (list(range(3, 37, 3))) #create list from 3 to 36 step 3
shape = (10, 5, 5)
data_bal = np.ones(shape) #create array to fill with features
data_bal = np.expand_dims(data_bal, axis=0) #expand one dimension to concatenate
data_lab = np.ones(1) #create array to fill labels
for i in num:
  indices = np.where((labels > i-3) & (labels <= i)) #select indcies from every 3 meter interval until 36
  sampled_indices = np.random.choice(indices[0].flatten(), size=sample_size, replace=False) #random sample of each interval
  tempx = features[sampled_indices]
  tempy = labels[sampled_indices]
  data_bal = np.concatenate((data_bal, tempx), axis=0)
  data_lab = np.concatenate((data_lab, tempy), axis=0)

indices = np.where((labels > 36)) #add the values > 36 m, they are so few no sample needed
sampled_indices = indices[0].flatten()
tempx = features[sampled_indices]
tempy = labels[sampled_indices]
data_bal = np.concatenate((data_bal[1:], tempx), axis=0)
data_lab = np.concatenate((data_lab[1:], tempy), axis=0)
data_bal2 = data_bal
data_bal2[data_bal2 > ceiling] = ceiling
data_bal2 = (data_bal2 - np.mean(data_bal2)) / np.std(data_bal2)** (1/2)


In [155]:
data_bal2.shape

(10488, 10, 5, 5)

## Train test split

In [158]:
Xtrain, Xtest, ytrain, ytest = train_test_split(data_bal, data_lab , test_size = 0.3, random_state=3) #create train, test set

## Train and evaluate Cnn

In [58]:
%load_ext tensorboard

In [ ]:
modelCnn = Sequential() #bulid cnn
modelCnn.add(InputLayer(input_shape = (10, 5, 5)))
modelCnn.add(Conv2D(filters=50, kernel_size= (3,3), strides=  1 , padding = "valid", activation='relu' )) #padding valid
#modelCnn.add(Conv2D(filters=100, kernel_size= (3,3), strides=  1 , padding = "valid", activation='relu' ))
modelCnn.add(MaxPool2D(pool_size = (2,2)))
modelCnn.add(Flatten())
modelCnn.add(Dense(400, activation='relu'))
modelCnn.add(Dense(200, activation='relu'))
#modelCnn.add(Dropout(0.25))
modelCnn.add(Dense(100, activation='relu'))
modelCnn.add(Dropout(0.4))
modelCnn.add(Dense(1, activation='linear'))

modelCnn.summary()




In [ ]:
modelCnn.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_percentage_error']) #compile cnn
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

model_save = ModelCheckpoint("/content/gdrive/MyDrive/NNmodels/best_modelCnnSmallStdBal.hdf5", save_best_only = True) #save best model

modelCnn.fit(Xtrain, ytrain, epochs = 100, validation_data=(Xtest, ytest), callbacks=[tensorboard_callback, model_save]) #train cnn  batch_size = 32

In [227]:
bmodel = load_model('/content/gdrive/MyDrive/NNmodels/best_modelCnnSmallStdBal.hdf5')
ypred_nn = bmodel.predict(Xtest) #predict best model


mse_nn = mse(ytest, ypred_nn) #calculate metrics
rmse_nn = mse_nn ** (1/2)
mae_nn = mae(ytest, ypred_nn)
mape_nn = mape(ytest, ypred_nn)

print(mape_nn)
print(mae_nn)
print(rmse_nn)

99/99 [==============================] - 1s 7ms/step
0.4586004335747886
6.028142741569686
8.379354940878384


In [ ]:
%tensorboard --logdir logs

In [ ]:
mean_nn = np.mean(ypred_nn[:]) #calculate mean
quantiles_nn = np.percentile(ypred_nn[:], [1, 25, 50, 75, 99]) #calculate quantiles 0.01, 0.25, 0.5, 0.75, 0.99
mean_labels = np.mean(data_lab[:])
quantiles_labels = np.percentile(data_lab[:], [1, 25, 50, 75, 99])

print(mean_nn)
print(quantiles_nn)
print(np.sort(ypred_nn.flatten())[:10]) #print the 10 lowest predictions
print(np.sort(ypred_nn.flatten())[-10:][::-1]) #print the 10 highest predictions

print(mean_labels)
print(quantiles_labels)
print(np.sort(data_lab.flatten())[:10])
print(np.sort(data_lab.flatten())[-10:][::-1])

## Predict image with sliding window

In [ ]:
tmodel = load_model('/content/gdrive/MyDrive/NNmodels/best_modelCnnSSmall.hdf5')

In [ ]:
Xs = np.load('/content/images/image_000.npy') #takes a long time depending on the size of the image
#Xsc = Xs[:, :64, 1024-64 :] #crop smaller subimage
Xsc = (Xsc - np.mean(Xsc)) / np.std(Xsc)** (1/2) #normalize features
height, width = Xsc.shape[1], Xsc.shape[2] #get size of image
window_size = 5
predicted_image = np.ones((1, height, width)) #create image to fill

for h in range(height - window_size + 1):
    for w in range(width - window_size + 1):
        window = Xsc[:, h:h+window_size, w:w+window_size]
        window_reshaped = np.expand_dims(window, axis=0)  # Reshape the window to match the expected input shape
        ypred = tmodel.predict(window_reshaped)

        # Assign the predicted value to the corresponding position in the predicted_image array
        predicted_image[:, h, w] = ypred
